# Autograd: 자동 미분
---------------------------
PyTorch의 모든 뉴럴 네트워크의 중심은 ``autograd`` 패키지가 있습니다.

``Autograd`` 패키지는 Tensors의 모든 작업에 대해 자동 미분을 제공합니다. 이것은 define-by-run 프레임워크입니다. 다시말해 코드를 어떻게 작성하느냐에 따라 역전파(backprop)가 정의된다는 뜻이며, 학습 과정의 매 단계(iteration)마다 역전파는 달라집니다.

## Tensor

``torch.Tensor``는 패키지의 코어 클래스입니다. 속성 ``.requires_grad``를 ``True``로 설정하면, 텐서의 모든 작업을 추적하기 시작합니다. 계산이 끝나면 ``.backward()``를 호출하여 모든 그래디언트를 자동으로 계산할 수 있습니다. 이 텐서의 그래디언트는 ``.grad`` 속성에 누적됩니다.

텐서가 히스토리를 추적하는 것을 막으려면 ``.detach()``를 호출하여 연산 히스토리에서 이를 분리하고 미래의 계산을 추적하지 못하게 할 수 있습니다.

추적 기록 (및 메모리 사용)을 방지하기 위해 ``torch.no_grad():`` 로 코드 블록을 래핑 할 수도 있습니다. 이는 특히 모델 평가에 유용합니다. 왜냐하면 평가할 모델은 이미 ``requires_grad=True``로 설정되어 학습 가능한 파라미터들을 갖고 있지만 그래디언트는 필요 없기 때문입니다.

autograd 구현을 위해 매우 중요한 클래스가 하나 더 있습니다. 바로 ``Function`` 입니다.

``Tensor``와 ``Function``은 서로 연결되어 있으며 비순환 그래프를 이루는데 이 그래프는 계산 기록 전체에 대하여 인코딩을 합니다. 각 텐서는 ``.grad_fn`` 속성을 가지고 있습니다. 이 속성은 텐서를 만든 ``Function``을 참조합니다. (단, 사용자가 만든 텐서는 예외입니다. 이 텐서는 ``grad_fn이 None``입니다)

편미분을 계산하고 싶다면 ``Tensor``에 있는 ``.backward()``를 호출하면 됩니다. 만약 Tensor가 스칼라면(예를들어, 하나의 element 값만 갖는 경우) ``backward()``의 argument에 인자를 넣어줄 필요가 없지만 스칼라가 아닌 경우에는 shape이 일치하는 텐서형인 ``gradient`` 인수를 지정해야합니다 .

In [1]:
import torch

텐서를 만들고 이것의 연산을 추적하기 위해 ``requires_grad=True`` 로 설정합니다

In [3]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


텐서에 연산을 수행합니다

In [4]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y는 연산 결과이며 ``grad_fn`` 속성을 가지고 있습니다

In [5]:
print(y.grad_fn)

y에 더 많은 연산을 실행합니다

In [7]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


``.requires_grad_( ... )``는 이미 존재하는 텐서의 ``requires_grad 플래그 값``을 변경합니다. 값을 명시하지 않을 경우 디폴트는 ``True``입니다

In [8]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


## Gradients

이제 backprop을 해봅시다. out 변수는 단일 스칼라 이기 때문에 ``out.backward()``는 ``out.backward(torch.tensor(1.))``과 동일한 결과를 가져옵니다

In [10]:
out.backward()

이제 ``d(out)/dx`` 그래디언트를 출력해봅시다

In [11]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


결과로 ``4.5``를 얻어야만 합니다. 여기서 ``out``텐서를 "$o$"라 해본다면

$o = \frac{1}{4}\sum_i z_i$ 이며 $z_i = 3(x_i+2)^2$ 이고
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$라면 따라서 $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$이 성립합니다

수학적으로 만약에 우리가 벡터 함수 $\vec{y}=f(\vec{x})$ 가 있다면 $\vec{x}$에 대한 $\vec{y}$의 그래디언트는 Jacobian matrix 입니다

$
J=\left(\begin{array}{ccc}
 \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
 \vdots & \ddots & \vdots\\
 \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
 \end{array}\right)
$

일반적으로 말해서 ``torch.autograd``는 벡터-야코비안 곱을 계산하는 엔진입니다. 즉 어떤 벡터 $v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$에 대해 $v^{T}\cdot J$를 연산합니다. 만약 $v$가 스칼라 함수 $l=g\left(\vec{y}\right)$ 의 기울기인 경우, $v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$ 이며, 연쇄법칙에 따라 벡터-야코비안 곱은 $\vec{x}$에 대한 $l$의 기울기가 됩니다

$
   J^{T}\cdot v=\left(\begin{array}{ccc}
    \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
    \vdots & \ddots & \vdots\\
    \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
    \end{array}\right)\left(\begin{array}{c}
    \frac{\partial l}{\partial y_{1}}\\
    \vdots\\
    \frac{\partial l}{\partial y_{m}}
    \end{array}\right)=\left(\begin{array}{c}
    \frac{\partial l}{\partial x_{1}}\\
    \vdots\\
    \frac{\partial l}{\partial x_{n}}
    \end{array}\right)
$

(여기서 $v^{T}\cdot J$ 은 $J^{T}\cdot v$ 를 취했을 때의 열 벡터로 취급할 수 있는 행 벡터를 갖습니다.) 벡터-야코비안 곱의 이러한 특성은 스칼라가 아닌 출력을 갖는 모델에 외부 변화도를 제공(feed)하는 것을 매우 편리하게 해줍니다.

이제 벡터-야코비안 곱의 예제를 살펴보도록 하겠습니다:

In [13]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([   79.4761,   957.2360, -1023.0236], grad_fn=<MulBackward0>)


이 경우 ``y`` 는 더 이상 스칼라 값이 아닙니다. ``torch.autograd`` 는 전체 야코비안을 직접 계산할수는 없지만, 벡터-야코비안 곱은 간단히 ``backward`` 에 해당 벡터를 인자로 제공하여 얻을 수 있습니다:

In [14]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


또한 ``with torch.no_grad():`` 로 코드 블럭을 감싸서 autograd가 ``.requires_grad=True`` 인 Tensor들의 연산 기록을 추적하는 것을 멈출 수 있습니다.

In [15]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


또는 ``.detach()`` 를 호출하여 내용물(content)은 같지만 require_grad가 다른 새로운 Tensor를 가져옵니다:

In [16]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
